In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import json

In [2]:
data_dir = "/media/mountHDD2/khoibaocon"
print(os.listdir(data_dir))

['TrainingSet3', 'Label.csv', 'alldata', 'TrainingSet1', 'single_label.csv', 'TrainingSet2']


In [3]:
main_df = pd.read_csv(data_dir + "/Label.csv")
main_df.shape

(6877, 4)

In [4]:
main_df.head()

,Recording,First_label,Second_label,Third_label
0,A0001,5,NaN,NaN
1,A0002,1,NaN,NaN
2,A0003,2,NaN,NaN
3,A0004,2,NaN,NaN
4,A0005,7,NaN,NaN


In [5]:
main_df["First_label"].value_counts()

First_label
5    1695
2    1098
1     918
8     826
3     704
7     653
6     574
4     207
9     202
Name: count, dtype: int64

In [6]:
main_df["Second_label"].value_counts()

Second_label
5.0    162
2.0    123
7.0     47
6.0     42
8.0     41
4.0     28
3.0     18
9.0     16
Name: count, dtype: int64

In [7]:
main_df["Third_label"].value_counts()

Third_label
9.0    2
8.0    2
6.0    1
4.0    1
Name: count, dtype: int64

In [8]:
single_main_df = main_df[main_df["Second_label"].isnull()]
single_main_df.shape

(6400, 4)

In [9]:
mat_files = glob(data_dir + "/alldata/*")
print(len(mat_files))

6877


In [10]:
single_fns = single_main_df["Recording"].values.tolist()
print(len(single_fns))

6400


In [11]:
single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]
print(os.path.exists(single_mat_paths[0]))

True


In [12]:
sample_data = loadmat(single_mat_paths[0])
sample_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'ECG'])

In [13]:
sample_signal_data = sample_data['ECG'][0][0][2]
sample_signal_data.shape

(12, 7500)

In [14]:
sample_sig = torch.randn(1, 12, 32)
conv_test = nn.Conv1d(12, 12, 3, 1, 1)
print(conv_test(sample_sig).shape)

torch.Size([1, 12, 32])


In [15]:
class BasicBlock(nn.Module):
    def __init__(self, channel_num):
        super(BasicBlock, self).__init__()
        self.conv_block1 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
			nn.ReLU(),
		)
        self.conv_block2 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
		)
        self.relu = nn.ReLU()
        torch.nn.init.kaiming_normal_(self.conv_block1[0].weight)
        torch.nn.init.kaiming_normal_(self.conv_block2[0].weight)
        
    def forward(self, x):
        residual = x
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x + residual
        out = self.relu(x)
        return out

In [16]:
test_basic_block = BasicBlock(2)
sample_sig = torch.randn(1, 2, 32)
print(test_basic_block(sample_sig).shape)

torch.Size([1, 2, 32])


In [17]:
class ResNet(nn.Module):
    def __init__(self, in_channels = 12, type = 18, num_classes = 9):
        super(ResNet, self).__init__()
        self.struc_dict = {
            18: {
                "num_channels" : [64, 128, 256, 512],
                "counts" : [2, 2, 2, 2]
            }
        }
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2)
        torch.nn.init.kaiming_normal_(self.conv1.weight)
        self.max1 = nn.MaxPool1d(kernel_size=3, stride=2)
        self.main = nn.Sequential()
        for idx, struc in enumerate(
            zip(
                self.struc_dict[type]["num_channels"], 
                self.struc_dict[type]["counts"]
            )
        ):
            num_channel, cnt = struc
            for i in range(cnt):
                self.main.add_module(f"conv{idx+1}_{i}", BasicBlock(num_channel))
            if idx < len(self.struc_dict[type]["num_channels"]) - 1:
                self.main.add_module(f"ext_{idx}", nn.Conv1d(num_channel, self.struc_dict[type]["num_channels"][idx+1], 3, 1))
                self.main.add_module(f"extbn_{idx}", nn.BatchNorm1d(self.struc_dict[type]["num_channels"][idx+1]))
                                     
        self.avg = torch.nn.AdaptiveAvgPool1d((1))
        self.lin = nn.Linear(self.struc_dict[type]["num_channels"][-1], num_classes)
        torch.nn.init.kaiming_normal_(self.lin.weight)
    def forward(self, x):
        x = self.conv1(x)
        x = self.max1(x)
        x = self.main(x)
        x = self.avg(x)
        x = x.reshape(x.shape[0], -1)
        x = self.lin(x)
        return x

In [18]:
model = ResNet()
sample_sig = torch.randn(1, 12, 3000)
model(sample_sig).shape

torch.Size([1, 9])

In [19]:
class ECG(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        data = loadmat(data_path)['ECG'][0][0][2]
        clip_data = data[:, 300:3000]

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["Recording"] == filename]["First_label"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label-1

    def __len__(self):
        return len(self.data_paths)

In [20]:
check_ds = ECG(data_paths=single_mat_paths, label_df=single_main_df)
sample, lbl = check_ds[0]
print(sample.shape, lbl)

torch.Size([12, 2700]) 4


In [21]:
model(sample.unsqueeze(dim=0)).shape

torch.Size([1, 9])

In [22]:
data_dict = {
    idx : [] for idx in range(9)
}

for data_path in single_mat_paths:
    filename = data_path.split("/")[-1].split(".")[0]
    _cls = single_main_df[single_main_df["Recording"] == filename]["First_label"].values.item()

    data_dict[_cls-1].append(data_path)

for key in data_dict:
    print(f"{key}->{len(data_dict[key])}")

0->918
1->976
2->686
3->179
4->1533
5->532
6->607
7->784
8->185


In [23]:
train_data_dict = {
    _cls : data_dict[_cls][:int(0.9*len(data_dict[_cls]))] for _cls in data_dict
}

valid_data_dict = {
    _cls : data_dict[_cls][int(0.9*len(data_dict[_cls])):] for _cls in data_dict
}

for key in train_data_dict:
    print(f"{key}->{len(train_data_dict[key])}--{len(valid_data_dict[key])}")

0->826--92
1->878--98
2->617--69
3->161--18
4->1379--154
5->478--54
6->546--61
7->705--79
8->166--19


In [24]:
train_data_paths = []
for key in train_data_dict:
    train_data_paths.extend(train_data_dict[key])
valid_data_paths = []
for key in valid_data_dict:
    valid_data_paths.extend(valid_data_dict[key])
print(len(train_data_paths))
print(len(valid_data_paths))

5756
644


In [25]:
train_ds = ECG(train_data_paths, single_main_df)
valid_ds = ECG(valid_data_paths, single_main_df)

print(len(train_ds))
print(len(valid_ds))

5756
644


In [26]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = os.getcwd() + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

    
def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
# batch_size = 32
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

print(len(traindl))
print(len(validdl))

90
644


In [28]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)

    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()

In [29]:
save_dir = "/home/linhpika/git/HeartResearch/Experiment/Approach/Model_signal/Focal_Loss"
checkpoint_folder = "run_resnet_gamma3_lr0.001_9-1"

In [30]:
# epoch = 100
epoch = 150
# lr = 0.0001 - 0.79
# lr = 0.001 - 0.809
lr = 0.001
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch)
loss_fn = nn.CrossEntropyLoss()

best_acc = 0
best_ep = 0
class_la = [1,2,3,4,5,6,7,8,9]
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23

In [31]:
# Training
for e in range(epoch):
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            
            valid_sig = valid_sig.to(device)  
            pred = model(valid_sig)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            

            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
#         checkpoint(valid_class_acc = val_correct, 
#                    val_total_loss = val_total_loss,
#                    old_valid_class_acc = old_valid_class_acc ,
#                    old_valid_loss = old_valid_loss,
#                    epoch = e, 
#                    model = model,
#                    optimizer = optimizer,
#                    check_folder = checkpoint_folder)
        if val_correct >= old_valid_class_acc and val_total_loss <= old_valid_loss:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }

         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()
# print(type(y_true))
print(y_true)
print(f"pred: {pred}")

# Save loss and acc to json file
acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
# Save plot acc and loss
# loss_plot(train_losses = train_losses, 
#           val_losses = val_losses, n_epochs = e , 
#           check_folder = checkpoint_folder)
# acc_plot(train_cls_acc = train_acc_plot , val_cls_acc = val_acc_plot, 
#                   n_epochs = e, check_folder = checkpoint_folder)

# AUC
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)
# print(fpr)
# print(f"tpr: {tpr}")
auc1 = metrics.auc(fpr, tpr)

# print(f"hihi {auc1}")
# print(type(auc1))

# Classification report
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])

reports = classification_report(y_true, pred, target_names=class_la, output_dict=True) 
# print(reports)
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

# json_object = json.dumps(reports, indent = 4) 
# print(json_object)



# ROC Curve
# fpr = [0] * 9
# tpr = [0] * 9
# thresholds = [0] * 9
# auc_score = [0] * 9
 
# for i in range(9):
#     fpr[i], tpr[i], thresholds[i] = roc_curve(y_true[:, i],
#                                               pred[:, i])
#     auc_score[i] = auc(fpr[i], tpr[i])
# print(auc_score)
# for i in range(9):
#     r2 = roc_auc_score(y_true, pred, average='micro', multi_class='ovr')
#     print("The ROC AUC score of "+ class_la[i] +" is: "+str(r2))

                    # Compute ROC curve and ROC area for each class
# class_fpr = {}
# class_tpr = {}
# class_roc_auc = dict()
# for i in range(9):
#     class_fpr[i], class_tpr[i], _ = roc_curve(valid_label[:, i].cpu(), pred[:, i].cpu(), drop_intermediate=False)
#     roc_auc[i] = auc(class_fpr[i], class_tpr[i])

# now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
# run_dir = save_dir + f"/{check_folder}"
# if not os.path.exists(run_dir):
#     os.mkdir(run_dir)
# save_ROC_dir = run_dir + f"/save_ROC_{now}"
# if not os.path.exists(save_ROC_dir):
#     os.mkdir(save_ROC_dir)

# plt.plot(class_fpr[0], class_tpr[0],'turquoise',label='1: ROC curve of class 1 (area = %0.2f)' % roc_auc[0])
# plt.plot(class_fpr[1], class_tpr[1],'peachpuff',label='2: ROC curve of class 2 (area = %0.2f)' % roc_auc[1])
# plt.plot(class_fpr[2], class_tpr[2],'paleturquoise',label='3: ROC curve of class 3 (area = %0.2f)' % roc_auc[2])
# plt.plot(class_fpr[3], class_tpr[3],'pink',label='4: ROC curve of class 4 (area = %0.2f)' % roc_auc[3])
# plt.plot(class_fpr[4], class_tpr[4],'lightcoral',label='5: ROC curve of class 5 (area = %0.2f)' % roc_auc[4])
# plt.plot(class_fpr[5], class_tpr[5],'peachpuff',label='6: ROC curve of class 6 (area = %0.2f)' % roc_auc[5])
# plt.plot(class_fpr[6], class_tpr[6],'steelblue',label='7: ROC curve of class 7 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[7], class_tpr[7],'forestgreen',label='8: ROC curve of class 8 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[8], class_tpr[8],'darkslategray',label='9: ROC curve of class 9 (area = %0.2f)' % roc_auc[6])

# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([-0.1, 1.1])
# plt.ylim([-0.1, 1.1])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic of lead')
# plt.legend(loc="lower right")
# plt.savefig(save_ROC_dir)



Epoch: 1


90it [00:14,  6.05it/s]

train loss: 12791.814191839669 - train acc: 15.201528839471855



644it [00:04, 158.01it/s]

valid loss: 2.070455693901976 - valid acc: 18.478260869565215
Epoch: 2


...


90it [00:14,  6.41it/s]

train loss: 18.71756782960356 - train acc: 74.49617790132037



644it [00:04, 158.40it/s]

valid loss: 6.962900725076001 - valid acc: 9.782608695652174
Best acuracy: 0.7763975155279503 at epoch 100
[4 4 3 6 4 4 4 7 8 0 4 7 7 7 7 0 2 7 4 2 8 2 1 1 4 2 2 0 1 2 2 0 4 8 6 4 7
 1 2 0 5 1 2 5 4 4 4 4 0 0 5 7 2 4 7 5 6 1 1 7 5 2 1 0 0 5 5 0 5 7 0 2 7 4
 4 4 4 1 6 4 7 2 1 2 2 6 3 0 0 2 2 0 1 3 4 1 4 0 4 4 5 8 1 7 4 4 2 4 5 7 2
 6 2 4 7 0 4 6 7 4 1 5 1 7 3 4 5 0 1 1 7 2 6 1 4 1 7 5 2 8 5 0 4 0 3 7 0 6
 0 7 4 7 0 8 6 4 6 0 4 4 4 1 5 1 4 3 4 7 2 0 5 5 2 5 7 4 6 4 2 1 5 1 0 6 1
 1 4 4 1 1 2 7 1 7 4 2 8 0 4 4 7 6 1 7 2 4 5 1 4 8 0 1 4 2 4 1 7 1 0 4 5 7
 5 2 4 4 4 4 1 1 7 2 7 1 0 7 4 6 1 1 4 0 0 0 1 4 5 6 5 4 4 2 4 5 7 0 7 1 2
 5 2 3 0 1 0 0 4 6 7 4 4 1 4 5 4 0 8 0 0 0 4 4 2 6 7 4 7 4 0 4 4 7 7 4 7 1
 2 1 5 4 4 1 7 4 4 6 5 4 6 4 1 5 1 1 6 7 0 0 0 1 0 0 6 6 2 6 3 0 2 2 4 4 4
 4 6 0 4 1 4 4 1 2 8 5 5 6 4 2 7 6 2 6 0 6 4 7 0 3 4 7 2 4 4 4 5 0 5 3 1 6
 4 6 1 0 7 4 1 2 2 1 4 5 7 7 7 1 6 4 4 4 1 2 6 5 1 1 4 0 1 0 7 0 6 6 7 6 8
 0 6 4 7 2 5 2 4 7 0 3 2 1 4 6 6 0 1 8 5 2 5 4 7 5 7 4 0 2 4 4 4 1 5


/home/linhpika/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/linhpika/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/linhpika/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        
        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

Epoch: 0


90it [00:13,  6.43it/s]

train loss: 29.9926096776898 - train acc: 61.22307157748437



644it [00:05, 118.92it/s]

valid loss: 1.1996317307057778 - valid acc: 69.09937888198758
Epoch: 1


...

valid loss: 0.8181594309019009 - valid acc: 76.3975155279503
Epoch: 149



90it [00:06, 13.62it/s]

train loss: 6.021617072351863 - train acc: 84.8679638637943



644it [00:01, 327.46it/s]

valid loss: 0.7616854136604033 - valid acc: 75.93167701863354
Best acuracy: 0.7841614906832298 at epoch 44
